In [1]:
import numpy as np
from scipy.stats import rv_continuous

In [43]:
class CustomDistribution(rv_continuous):
    def rvs(self, *args, **kwargs):
        if np.random.rand() < 0.5:
            return None
        else:
            return np.random.normal(*args, **kwargs)

custom_dist = CustomDistribution(name='custom_dist')

In [56]:

custom_dist.rvs(size=10,loc=2,scale=0.1)


array([2.04302104, 1.99185057, 2.01461519, 1.97196757, 1.82350311,
       2.03865219, 2.00428072, 2.05048621, 1.99431688, 1.96660916])

In [1]:
import numpy as np
from scipy.stats import rv_continuous

class RandomRatio(rv_continuous):
    def __init__(self, ratio, *args, **kwargs):
        self.low = kwargs.pop('low', 0)
        self.high = kwargs.pop('high', 1)
        self.ratio = ratio
        super().__init__(*args, **kwargs)

    def rvs(self, *args, **kwargs):
        if np.random.rand() < 0.1:
            return None
        else:
            return self.ratio + np.random.uniform(self.low, self.high, *args, **kwargs)

custom_dist = RandomRatio(ratio=0.5, low=-.5, high=.5)


In [2]:
import pandas as pd

def balance_train_and_test(notcb_train:pd.DataFrame, cb_train:pd.DataFrame, notcb_test:pd.DataFrame, cb_test:pd.DataFrame, random_state:int):
    """
    This functions does not relabel the data that is moved!
    """
    x1 = notcb_train.shape[0]
    x2 = cb_train.shape[0]
    y1 = notcb_test.shape[0]
    y2 = cb_test.shape[0]

    x_hat = (y2 * x1 - x2 * y1) / (y2 + x2)

    
    if x_hat < 0: # take from the test and add to training
        np.testing.assert_allclose(ratio:=(y1 + x_hat)/(y2), (x1 - x_hat)/(x2), atol=1e-5)
        notcb_test = notcb_test.sample(frac=1, random_state=random_state)
        x_hat *= -1 
        data_to_move = notcb_test[:int(x_hat)]
        notcb_test = notcb_test[int(x_hat):]
        train_df = pd.concat([notcb_train, cb_train, data_to_move])
        test_df = pd.concat([notcb_test, cb_test])

    else: # take from training, add to test
        np.testing.assert_allclose(ratio:=(y1 + x_hat)/(y2), (x1 - x_hat)/(x2), atol=1e-5)
        notcb_train = notcb_train.sample(frac=1, random_state=random_state)
        data_to_move = notcb_train[:int(x_hat)]
        notcb_train = notcb_train[int(x_hat):]
        train_df = pd.concat([notcb_train, cb_train])
        test_df = pd.concat([notcb_test, cb_test, data_to_move])
    
    return train_df, test_df, x_hat, ratio # the new ratio of noncb to cb (in both sets)

In [4]:
import pandas as pd
import numpy as np
from pathlib import Path

from regex import B

path_to_data = Path('C:/Users/rooty/UWEC/Research/CyberBullyingML/venv/cyberbullying-ml/data/en_only')
TRAIN_DATA_NAME = '48000_cyberbullying_tweets_basic_clean.csv' 
TEST_DATA_NAME = 'hatespeech_tweets_basic_clean.csv'
RANDOM_SEED = 115

train_df = pd.read_csv(path_to_data / TRAIN_DATA_NAME)
test_df = pd.read_csv(path_to_data / TEST_DATA_NAME)
train_df.dropna(inplace=True)
train_df.drop_duplicates(inplace=True)
train_df.reset_index(drop=True, inplace=True)
test_df.dropna(inplace=True)
test_df.drop_duplicates(inplace=True)
test_df.reset_index(drop=True, inplace=True)
test_df.rename(columns={'class': 'label'}, inplace=True)
# relabel the data such that the offensive class is cyberbullying and the nonoffensive class is not cyberbullying (the hatespeech class is not included)
train_df['label'] = train_df['label'].apply(lambda x: 0 if x == 'notcb' else 1)
test_df['label'] = test_df['label'].map({0: 2, 1: 1, 2: 0})
# Balance the datasets
notcb_train = train_df[train_df['label'] == 0]
cb_train = train_df[train_df['label'] == 1]
notcb_test = test_df[test_df['label'] == 0]
cb_test = test_df[test_df['label'] == 1]
balanced_train, balanced_test, x_hat, ratio = balance_train_and_test(notcb_train, cb_train, notcb_test, cb_test, random_state=RANDOM_SEED)

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

def pp(text):
    return ''.join([char for char in text if ord(char) < 128])

cv = CountVectorizer(analyzer='char_wb', ngram_range=(1,3), preprocessor=pp)
cv.fit(balanced_train['tweet'])
x_train = cv.transform(balanced_train['tweet'])
x_test = cv.transform(balanced_test['tweet'])
y_train = balanced_train['label']
y_test = balanced_test['label']

In [15]:
cv.get_feature_names_out()

array([' ', ' 0', ' 0 ', ..., 'zzw', 'zzy', 'zzz'], dtype=object)

In [16]:
import xgboost as xgb

m = xgb.XGBClassifier()
m.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [17]:
from sklearn.metrics import classification_report

# Perform predictions on x_test
y_pred = m.predict(x_test)

# Print the classification report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.63      0.61      0.62      3243
           1       0.93      0.93      0.93     17476

    accuracy                           0.88     20719
   macro avg       0.78      0.77      0.77     20719
weighted avg       0.88      0.88      0.88     20719



In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem.snowball import SnowballStemmer
from xgboost import XGBClassifier
import itertools


snowballer = SnowballStemmer('english')

def pp_SnowballStemmer(text):
    return ' '.join([snowballer.stem(word) for word in text.split()])


In [35]:


xgb_param_grid = {
    'vectorizer': [CountVectorizer],
    'vectorizer__ngram_range': [(1,1), (1,3)],
    'vectorizer__preprocessor': [pp_SnowballStemmer, None],
    'vectorizer__max_df': [0.5, 0.75],
    'classifier__learning_rate': [0.01, 0.1],
    'classifier__n_estimators': [100, 500],
    'classifier__colsample_bytree': [0.5, 0.75],
    'classifier__max_depth': [6, 12],
    'classifier': [XGBClassifier]
}
iters = (v for _, v in xgb_param_grid.items())
keys = xgb_param_grid.keys()
param_sets = [params for params in itertools.product(*iters)]

In [36]:
# best: 47, 31, 77
len(param_sets)

128

In [2]:
import pandas as pd
df = pd.read_csv('C:/Users/rooty/UWEC/Research/CyberBullyingML/cyberbullyingml/cyberbullying-ml/data/en_only/48000_cyberbullying_tweets_basic_clean.csv')
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df['label'] = df['label'].apply(lambda x: 0 if x == 'notcb' else 1)
df.head()


,tweet,label
0,Here at home Neighbors pick on my family and I...,1
1,Being bullied at school Highachieving boys use...,1
2,There was a girl in my class in 6th grade who ...,1
3,He is probably a white gay kid from some subur...,1
4,You are pushed ti resorting Treating thr bulli...,1


In [4]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(ngram_range=(1,3), max_df=0.9)


In [7]:
x = cv.fit_transform(df['tweet'])
y = df['label']

In [1]:
from sklearn.base import clone
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import StratifiedKFold, cross_validate

import xgboost as xgb

X, y = load_breast_cancer(return_X_y=True)


def fit_and_score(estimator, X_train, X_test, y_train, y_test):
    """Fit the estimator on the train set and score it on both sets"""
    estimator.fit(X_train, y_train, eval_set=[(X_test, y_test)])

    train_score = estimator.score(X_train, y_train)
    test_score = estimator.score(X_test, y_test)

    return estimator, train_score, test_score


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=94)

clf = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=3)

results = {}

for train, test in cv.split(X, y):
    X_train = X[train]
    X_test = X[test]
    y_train = y[train]
    y_test = y[test]
    est, train_score, test_score = fit_and_score(
        clone(clf), X_train, X_test, y_train, y_test
    )
    results[est] = (train_score, test_score)

[0]	validation_0-logloss:0.47435
[1]	validation_0-logloss:0.37961
[2]	validation_0-logloss:0.31334
[3]	validation_0-logloss:0.26883
[4]	validation_0-logloss:0.23208
[5]	validation_0-logloss:0.21337
[6]	validation_0-logloss:0.19236
[7]	validation_0-logloss:0.18360
[8]	validation_0-logloss:0.17274
[9]	validation_0-logloss:0.17064
[10]	validation_0-logloss:0.16569
[11]	validation_0-logloss:0.15752
[12]	validation_0-logloss:0.15063
[13]	validation_0-logloss:0.14838
[14]	validation_0-logloss:0.15396
[15]	validation_0-logloss:0.15414
[0]	validation_0-logloss:0.45802
[1]	validation_0-logloss:0.34767
[2]	validation_0-logloss:0.27301
[3]	validation_0-logloss:0.22877
[4]	validation_0-logloss:0.19318
[5]	validation_0-logloss:0.17110
[6]	validation_0-logloss:0.15218
[7]	validation_0-logloss:0.13440
[8]	validation_0-logloss:0.13049
[9]	validation_0-logloss:0.12544
[10]	validation_0-logloss:0.12489
[11]	validation_0-logloss:0.12767
[12]	validation_0-logloss:0.12369
[13]	validation_0-logloss:0.12046


In [5]:
clf.fit(X_train, y_train, eval_set=[(X_test, y_test)])

[0]	validation_0-logloss:0.45121
[1]	validation_0-logloss:0.34254
[2]	validation_0-logloss:0.26194
[3]	validation_0-logloss:0.22309
[4]	validation_0-logloss:0.18734
[5]	validation_0-logloss:0.15369
[6]	validation_0-logloss:0.13510
[7]	validation_0-logloss:0.12163
[8]	validation_0-logloss:0.10814
[9]	validation_0-logloss:0.09494
[10]	validation_0-logloss:0.08578
[11]	validation_0-logloss:0.07599
[12]	validation_0-logloss:0.07139
[13]	validation_0-logloss:0.06514
[14]	validation_0-logloss:0.06103
[15]	validation_0-logloss:0.05669
[16]	validation_0-logloss:0.05490
[17]	validation_0-logloss:0.05099
[18]	validation_0-logloss:0.04994
[19]	validation_0-logloss:0.04871
[20]	validation_0-logloss:0.04538
[21]	validation_0-logloss:0.04585
[22]	validation_0-logloss:0.04402
[23]	validation_0-logloss:0.04377
[24]	validation_0-logloss:0.04376
[25]	validation_0-logloss:0.04260
[26]	validation_0-logloss:0.04145
[27]	validation_0-logloss:0.04131
[28]	validation_0-logloss:0.04021
[29]	validation_0-loglos

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=3,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [6]:
booster = clf.get_booster()
print(booster.num_boosted_rounds())


43


In [2]:
from pathlib import Path
from sklearn.model_selection import train_test_split
import sys
import pandas as pd

sys.path.append(r'C:\Users\rooty\UWEC\Research\CyberBullyingML\cyberbullyingml\cyberbullying-ml\official\src')
from utils.params import get_topn_param_sets

DATA_PATH = Path(r'C:\Users\rooty\UWEC\Research\CyberBullyingML\cyberbullyingml\cyberbullying-ml\data\en_only')


TRAIN_DATA_NAME = '48000_cyberbullying_tweets_basic_clean.csv'
TEST_DATA_NAME = 'hatespeech_tweets_basic_clean.csv'
PARAMS_PATH = Path(r'C:\Users\rooty\UWEC\Research\CyberBullyingML\cyberbullyingml\cyberbullying-ml\official\hyp_search\results')
RANDOM_SEED = 115

In [8]:
import time
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
# Load dataset
train_df = pd.read_csv(DATA_PATH/TRAIN_DATA_NAME)

train_df.dropna(inplace=True)
train_df.drop_duplicates(inplace=True)
train_df.reset_index(drop=True, inplace=True)

X = train_df['tweet']
y = train_df['label']

y = y.apply(lambda label : 0 if label == 'notcb' else 1)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

cv = CountVectorizer()
X_train = cv.fit_transform(X_train)

# Initialize the classifier
clf = XGBClassifier(tree_method="hist", n_estimators=1000, learning_rate=0.001, n_jobs=-1)

start_time = time.time()
# Train the classifier
clf.fit(X_train, y_train)
end_time = time.time()

print(f'Total time: {end_time - start_time}') 

Total time: 17.468730926513672


In [37]:
# train_df = pd.read_csv(DATA_PATH / TRAIN_DATA_NAME)
# test_df = pd.read_csv(DATA_PATH / TEST_DATA_NAME)
# train_df.dropna(axis=0, inplace=True)
# train_df.drop_duplicates(inplace=True)
# train_df.reset_index(drop=True, inplace=True)
# test_df.dropna(axis=0, inplace=True)
# test_df.drop_duplicates(inplace=True)
# test_df.reset_index(drop=True, inplace=True)
# train_df['label'] = train_df['label'].apply(lambda label: 0 if label =='notcb' else 1)
# test_df['class'] = test_df['class'].apply(lambda label: 1 if label == 0 else 0)
# x_train, x_val, y_train, y_val = train_test_split(
#     train_df['tweet'], 
#     train_df['label'], 
#     test_size=0.1, 
#     stratify=train_df['label'],
#     random_state=RANDOM_SEED
# ) # for early stopping
# x_test = test_df['tweet']
# y_test = test_df['class']
catb_param_results:pd.DataFrame = get_topn_param_sets(PARAMS_PATH, algo='catb', dataset='d1', n=10, sort_condition='f1_macro_mean')

In [38]:
len(catb_param_results)

10

In [36]:
pd.set_option('display.max_colwidth', None) 
for i in range(len(catb_param_results)):
    print(catb_param_results['params'].iloc[i])

{'vectorizer': "<class 'sklearn.feature_extraction.text.CountVectorizer'>", 'vectorizer__ngram_range': [1, 3], 'vectorizer__preprocessor': None, 'vectorizer__max_df': 0.5, 'classifier__learning_rate': 0.1, 'classifier__n_estimators': 1000, 'classifier__rsm': 1, 'classifier__depth': 10, 'classifier': "<class 'catboost.core.CatBoostClassifier'>"}
{'vectorizer': "<class 'sklearn.feature_extraction.text.CountVectorizer'>", 'vectorizer__ngram_range': [1, 3], 'vectorizer__preprocessor': None, 'vectorizer__max_df': 0.9, 'classifier__learning_rate': 0.1, 'classifier__n_estimators': 1000, 'classifier__rsm': 1, 'classifier__depth': 10, 'classifier': "<class 'catboost.core.CatBoostClassifier'>"}
{'vectorizer': "<class 'sklearn.feature_extraction.text.CountVectorizer'>", 'vectorizer__ngram_range': [1, 1], 'vectorizer__preprocessor': None, 'vectorizer__max_df': 0.9, 'classifier__learning_rate': 0.1, 'classifier__n_estimators': 1000, 'classifier__rsm': 0.75, 'classifier__depth': 10, 'classifier': "<

In [3]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load sample data
data = load_iris()
X = data.data
y = data.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the XGBClassifier
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', early_stopping_rounds=1)

# Fit the model on the training data
model.fit(X_train, y_train, eval_set=[(X_test, y_test)])

# Make predictions on the test data



[0]	validation_0-mlogloss:0.72604
[1]	validation_0-mlogloss:0.50842
[2]	validation_0-mlogloss:0.36744
[3]	validation_0-mlogloss:0.27203
[4]	validation_0-mlogloss:0.20487
[5]	validation_0-mlogloss:0.15866
[6]	validation_0-mlogloss:0.12503
[7]	validation_0-mlogloss:0.09808
[8]	validation_0-mlogloss:0.08104
[9]	validation_0-mlogloss:0.06622
[10]	validation_0-mlogloss:0.05690
[11]	validation_0-mlogloss:0.04823
[12]	validation_0-mlogloss:0.04289
[13]	validation_0-mlogloss:0.03730
[14]	validation_0-mlogloss:0.03349
[15]	validation_0-mlogloss:0.02955
[16]	validation_0-mlogloss:0.02698
[17]	validation_0-mlogloss:0.02506
[18]	validation_0-mlogloss:0.02271
[19]	validation_0-mlogloss:0.02059
[20]	validation_0-mlogloss:0.01938
[21]	validation_0-mlogloss:0.01889
[22]	validation_0-mlogloss:0.01871
[23]	validation_0-mlogloss:0.01803
[24]	validation_0-mlogloss:0.01780
[25]	validation_0-mlogloss:0.01736
[26]	validation_0-mlogloss:0.01704
[27]	validation_0-mlogloss:0.01727


c:\Users\rooty\UWEC\Research\CyberBullyingML\cyberbullyingml\lib\site-packages\xgboost\core.py:158: UserWarning: [18:53:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=1,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [4]:
model.get_booster().num_boosted_rounds()

28

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

str(CountVectorizer(max_df=.2).__class__).split('.')[-1][:-2]


'CountVectorizer'

In [5]:
from pathlib import Path
PATH_TO_RESULTS = Path(r'C:\Users\rooty\UWEC\Research\CyberBullyingML\cyberbullyingml\cyberbullying-ml\official\experiments2.0\results')

import json

for file in PATH_TO_RESULTS.iterdir():
    with open(file, 'r') as f:
        data = json.load(f)
        results = data['results']
    for result in results:
        result['vectorizer'] = 'CountVectorizer' if 'CountVectorizer' in result['vectorizer'] else 'TfidfVectorizer'
        result['vectorizer_params']['preprocessor'] = None if result['vectorizer_params']['preprocessor'] == None else 'SnowballStemmer'
    with open(file, 'w') as f:
        json.dump(data, f, default=str)



In [10]:
from pathlib import Path
PATH_TO_RESULTS = Path(r'C:\Users\rooty\UWEC\Research\CyberBullyingML\cyberbullyingml\cyberbullying-ml\official\hyp_search\results')

import json

for file in PATH_TO_RESULTS.iterdir():
    with open(file, 'r') as f:
        print(file.__str__())
        data = json.load(f)
    print(data)

C:\Users\rooty\UWEC\Research\CyberBullyingML\cyberbullyingml\cyberbullying-ml\official\hyp_search\results\cv_search_catb_cv_d1__results.json
{'fit_time_mean': [33.03990920384725, 76.08390935262044, 34.25608944892883, 76.33992811044057, 8.599487741788229, 14.32163393497467, 8.536416927973429, 14.825903018315634, 34.06251001358032, 76.01266980171204, 34.362186431884766, 77.10689147313435, 8.418047626813253, 14.239410003026327, 8.535806973775228, 14.800995628039042, 50.79780705769857, 92.32040846347809, 50.694013357162476, 90.98667625586192, 14.14007318019867, 17.93074079354604, 12.167560617129007, 17.420497337977093, 51.22993163267771, 90.78522578875224, 50.268625577290855, 91.57244702180226, 14.098800500233969, 18.098105430603027, 12.137058138847351, 17.162883281707764, 604.0624193350474, 1327.7370247840881, 588.0029063224792, 1296.3743807077408, 138.79841820398966, 268.197389682134, 155.61401438713074, 245.50221820672354, 610.8936587174734, 1361.5952279170353, 591.6409598986307, 1324.5

In [1]:
import trace
import traceback

try:
    a = m['key']
except Exception as e:
    print(traceback.format_exc())

Traceback (most recent call last):
  File "C:\Users\rooty\AppData\Local\Temp\ipykernel_8744\2443630009.py", line 5, in <module>
    a = m['key']
NameError: name 'm' is not defined



In [4]:
import torch
from transformers import AutoModel, AutoTokenizer
from pathlib import Path
import pandas as pd



In [5]:
DATA_PATH = Path(r'C:\Users\rooty\UWEC\Research\CyberBullyingML\cyberbullyingml\cyberbullying-ml\data\en_only')

TRAIN_DATA_NAME = '48000_cyberbullying_tweets_basic_clean.csv'
TEST_DATA_NAME = 'hatespeech_tweets_basic_clean.csv'
RANDOM_SEED = 115

In [21]:
train_df = pd.read_csv(DATA_PATH/TRAIN_DATA_NAME)

train_df.dropna(inplace=True)
train_df.drop_duplicates(inplace=True)
train_df.reset_index(inplace=True, drop=True)

In [ ]:
bertweet = AutoModel.from_pretrained('vinai/bertweet-base')
tokenizer = AutoTokenizer.from_pretrained('vinai/bertweet-base')

In [46]:
tweet = "This is a test tweet"

enc_df = train_df.copy()

enc_df['tweet'] = enc_df['tweet'].apply(lambda tweet: tokenizer(tweet, return_tensors='pt'))
# NOTE pad encoded tweets??

In [48]:
enc_df['tweet'].iloc[0]

{'input_ids': tensor([[    0,   655,    35,   219, 41288,   979,    24,    23,   383,    13,
             8,  3218,    14,    23,   705,    17, 29050,    76,  1070,    43,
           485,   230,   236,   329,   148,  2235,  1545,   148,    19,    80,
           568,    13,  9478,   148,    48,     6,    78, 10012,  1463,    24,
            23,   989, 15903,   384,   106,   252,   201,     9,   148,     2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])}

In [38]:
x = torch.tensor(enc_df['tweet'].iloc[0]).unsqueeze(0)

In [43]:
bertweet(x)

AttributeError: 'BaseModelOutputWithPoolingAndCrossAttentions' object has no attribute 'logits'

In [19]:
enc_tweet

[0, 126, 17, 11, 1156, 10262, 1613, 738, 6618, 119, 2957, 855, 46358, 2]